In [34]:

Es = 1.
rms_slope = 1.
dx = 1  # pixel size

from SurfaceTopography.Generation import fourier_synthesis
import numpy as np
import matplotlib.pyplot as plt

# for the CM demo
from ContactMechanics import make_system
from ContactMechanics.Tools.Logger import screen

np.random.seed(0)

n = 256  # you can increase this to your convenience
s = n * dx

topo = fourier_synthesis(nb_grid_pts=(n, n),
                         hurst=0.8,
                         physical_sizes=(s, s),
                         rms_slope=1.,
                         long_cutoff=s / 2,
                         # decrease this to get closer to the
                         # thermodynamic limit (less fluctuations)
                         short_cutoff=dx * 4,
                         # increase for better discretization
                         # of the smallest wavelengths
                         )

# rescale the topography so that the rms slope is exactly one
# squeeze applies the scaling factor to the heights and save this as a copy
# this avoids that the scaling multiplication is applied at each call of height
print(topo.rms_gradient())
print(topo.rms_slope_from_profile())

topo = topo.scale(1 / topo.rms_gradient()).squeeze()

topo._heights = topo.heights() - np.max(topo.heights())


###############################################################################
####### DEMO of contact mechanics
###############################################################################

# lazy way to setup the system
system = make_system(substrate="periodic", surface=topo, young=Es)

1.0145420206083464
0.7016236250600787


In [35]:
gtol = 1e-6

for offset in [5*topo.rms_height_from_area() , 10 * topo.rms_height_from_area() ]:

    init_gap = - (topo.heights() + offset)
    init_gap[init_gap < 0] = 0

    init_disp = init_gap + (topo.heights() + offset)
    init_force = system.substrate.evaluate_force(init_disp)
    print("dual")
    for solver in ["l-bfgs-b", "ccg_without_restart", "ccg_with_restart"]:
        sol = system.dual_minimize_proxy(
            offset=offset,
            init_force=init_force,
            solver=solver,
            gtol=gtol
            #logger=screen
            )
        print(sol.message)
        print(f"solver: {solver}, nit={sol.nit}")
    print("contac area:", system.compute_contact_area() / n ** 2)
    print("primal")

    for solver in ["l-bfgs-b", "ccg_without_restart", "ccg_with_restart"]:
        sol = system.primal_minimize_proxy(
            offset=offset,
            init_gap=init_gap,
            solver=solver,
            gtol=gtol
            #logger=screen
            )
        print(sol.message)
        print(f"solver: {solver}, nit={sol.nit}")
    print("contac area:", system.compute_contact_area() / n ** 2)


dual
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
solver: l-bfgs-b, nit=76
it 2 : hessian is negative along the descent direction. You will probably need linesearch or trust region
CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL
solver: ccg_without_restart, nit=73
CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL
solver: ccg_with_restart, nit=161
contac area: 0.3651885986328125
primal
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
solver: l-bfgs-b, nit=76
CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL
solver: ccg_without_restart, nit=70
CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL
solver: ccg_with_restart, nit=187
contac area: 0.3651885986328125
dual
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
solver: l-bfgs-b, nit=93
it 2 : hessian is negative along the descent direction. You will probably need linesearch or trust region
CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL
solver: ccg_without_restart, nit=90
CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL
solver: ccg_with_restart, nit=116
contac area: 0.8812103271484375
primal
CONVERGEN

/Users/antoines/ContactMechanics/venv/lib/python3.8/site-packages/NuMPI/Optimization/ccg_with_restart.py:119: RuntimeWarning: invalid value encountered in double_scalars
  des_dir[mask_c] = -residual[mask_c] + delta * (G / G_old) * des_dir[


NO-CONVERGENCE: MAXITERATIONS REACHED
solver: ccg_with_restart, nit=999


AttributeError: 'NoneType' object has no attribute 'dtype'

In [ ]:
system.area_per_pt